## SICP 习题 （3.30）解题总结: 半加器和全加器

SICP 习题 3.30 让我们用3.28构建的逻辑门去定义半加器和全加器。

这里面的主要难点是对半加器和全加器的认识，属于逻辑电路课程中靠中间的内容。这里因为篇幅的原因就不展开描述了，这里倒是有个学习逻辑电路的建议：就是通过沙盒游戏《我的世界》去构建逻辑电路。

在《我的世界》的游戏里有个特殊的方块叫红石方块，可以当电线用，而且在《我的世界》里还可以通过岩石方块构建非门，与门，或门。

通过以上组件就可以在一个沙盒游戏里尝试各种逻辑电路了，当然也包括半加器和全加器。



以下是从3.28里拷贝过来的代码，包括一些解释，就原封不动放这里了。

在这些代码的后面是关于半加器和全加器的实现


首先是关于wire的函数，主要是用于建立连接，或者是一个“引脚”，可以往一个“引脚”发送信号，也可以获取里面的信号：

In [1]:
(define (make-wire)
  (let ((signal-value 0) (action-procedures '()))
    (define (set-my-signal! new-value)
      (if (not (= signal-value new-value))
	  (begin (set! signal-value new-value)
		 (call-each action-procedures))
	  'done))
    (define (accept-action-procedure! proc)
      (set! action-procedures (cons proc action-procedures))
      (proc))

    (define (dispatch m)
      (cond ((eq? m 'get-signal) signal-value)
	    ((eq? m 'set-signal!) set-my-signal!)
	    ((eq? m 'add-action!) accept-action-procedure!)
	    (else (error "Unknown operation --WIRE" m))))
    dispatch))

In [2]:
(define (get-signal wire)
  (wire 'get-signal))

(define (set-signal! wire new-value)
  ((wire 'set-signal!) new-value))



In [3]:

(define (call-each procedures)
  (if (null? procedures)
      'done
      (begin
	((car procedures))
	(call-each (cdr procedures)))))


以上定义了连线函数，或者说“引脚”函数，下面测试一下，先建立一个“引脚”，然后给它发射信号，然后获取里面的信号：

In [4]:
(define testing-wire (make-wire))

In [5]:
(get-signal testing-wire)

0

In [6]:
(set-signal! testing-wire 1)

done

In [7]:
(get-signal testing-wire)

1

然后我们定义两个“引脚”，是给一个inverter用的，inverter就是非门，输入1就会出0，输入0就出1.

下面两个引脚分别是inverter的输入和输出：

In [8]:
  (define inverter-input (make-wire))
  (define inverter-output (make-wire))

In [9]:
(get-signal inverter-input)

0

In [10]:
(get-signal inverter-output)

0

然后我们定义一个inverter函数，这个函数把input和output连接起来，里面定义了一个invert-input的内部函数，专门对输入进行转换，转换用的函数是(logical-not):

In [11]:
(define (inverter input output)
  (define (invert-input)
    (let ((new-value (logical-not (get-signal input))))
      (after-delay inverter-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! input invert-input)
  'ok)

(define (logical-not s)
  (cond ((= s 0 ) 1)
	((= s 1) 0)
	(else (error "Invalid signal" s))))

然后定义add-action!函数把一个动作绑定到“引脚”上：

In [12]:
(define (add-action! wire action-procedure)
  ((wire 'add-action!) action-procedure))


定义inverter操作的整体时延，这个非常重要，对于后续复杂线路的同步计算起到关键作用：

In [13]:
(define inverter-delay 2)

还有一个在指定“时间”后执行动作的函数。这里的“时间”线是通过一个列表来表示，(add-to-agenda!)函数会根据时延参数delay计算出具体要执行操作的时间点，然后挂载到agenda上：

In [14]:
(define (after-delay delay action)
  (add-to-agenda! (+ delay (current-time the-agenda))
		  action
		  the-agenda))

创建一个空的agenda

In [15]:
(define (make-agenda) (list 0))

In [16]:
(define the-agenda (make-agenda))

创建获取“当前时间”的函数，这里的“当前时间”是指在calendar列表里现在执行到那个“时间点”了：

In [17]:
(define (current-time agenda) (car agenda))

接着是关于calendar的实现：

In [18]:
(define (add-to-agenda! time action agenda)
  (define (belongs-before? segments)
    (or (null? segments)
	(< time (segment-time (car segments)))))
  (define (make-new-time-segment time action)
    (let ((q (make-queue)))
      (insert-queue! q action)
      (make-time-segment time q )))
  (define (add-to-segments! segments)
    (if (= (segment-time (car segments)) time)
	(insert-queue! (segment-queue (car segments))
		       action)
	(let ((rest (cdr segments)))
	  (if (belongs-before? rest)
	      (set-cdr!
	       segments
	       (cons (make-new-time-segment time action)
		     (cdr segments)))
	      (add-to-segments! rest)))))
  (let ((segments (segments agenda)))
    (if (belongs-before? segments)
	(set-segments!
	 agenda
	 (cons (make-new-time-segment time action)
	       segments))
	(add-to-segments! segments))))

In [19]:
(define (segments agenda) (cdr agenda))

calendar的实现依赖于队列，所以把队列代码也拷贝过来：

In [20]:
(define (make-queue) (cons '() '()))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue" queue)
      (car (front-ptr queue))))

(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
	   (set-front-ptr! queue new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue)
	  (else
	   (set-cdr! (rear-ptr queue) new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
	 (error "DELETE! called with an empty queue" queue))
	(else
	 (set-front-ptr! queue (cdr (front-ptr queue)))))
	 queue)

(define (print-queue queue)
  (cond ((null? queue)
	 (error "print-queue called with an incorrect data" queue))
	(else
	 (display (car queue)))))

(define (front-ptr queue) (car queue))

(define (rear-ptr queue) (cdr queue))

(define (set-front-ptr! queue item) (set-car! queue item))

(define (set-rear-ptr! queue item) (set-cdr! queue item))

(define (empty-queue? queue) (null? (front-ptr queue)))


相关的时间处理函数：

In [21]:
(define (make-time-segment time queue)
  (cons time queue))

(define (segment-time s) (car s))

(define (segment-queue s) (cdr s))

(define (set-current-time! agenda time)
  (set-car! agenda time))

(define (set-segments! agenda segments)
  (set-cdr! agenda segments))

(define (first-segment agenda) (car (segments agenda)))

(define (rest-segments agenda) (cdr (segments agenda)))

(define (empty-agenda? agenda)
  (null? (segments agenda)))

用于强制执行agenda里所有操作的函数：

In [22]:
(define (propagate)
  (if (empty-agenda? the-agenda)
      'done
      (let ((first-item (first-agenda-item the-agenda)))
	(first-item)
	(remove-first-agenda-item! the-agenda)
	(propagate))))

In [23]:
(define (first-agenda-item agenda)
  (if (empty-agenda? agenda)
      (error "Agenda is empty -- FIRST-AGENDA-ITEM")
      (let ((first-seg (first-segment agenda)))
	(set-current-time! agenda (segment-time first-seg))
	(front-queue (segment-queue first-seg)))))

(define (remove-first-agenda-item! agenda)
  (let ((q (segment-queue (first-segment agenda))))
    (delete-queue! q)
    (if (empty-queue? q)
	(set-segments! agenda (rest-segments agenda)))))

这样就可以开始测试了，创建一个inverter，就是一个非门：

In [24]:
(inverter inverter-input inverter-output)

ok

不做任何操作前先看看两个“引脚”的值：

In [25]:
(get-signal inverter-input)

0

In [26]:
(get-signal inverter-output)

0

给输入引脚一个1的信号：

In [27]:
(set-signal! inverter-input 1)

done

强制执行所有待处理操作：

In [28]:
(propagate)

done

再看看两个引脚的信号值：

In [29]:
(get-signal inverter-input)

1

In [30]:
(get-signal inverter-output)

0

反过来把输入引脚的信号设置成0

In [31]:
(set-signal! inverter-input 0)

done

In [32]:
(get-signal inverter-input)

0

In [33]:
(propagate)

done

再看看输出引脚的信号：

In [34]:
(get-signal inverter-output)

1

下面定义一个展示引脚信号的函数：

In [35]:

(define (probe name wire)
  (add-action! wire
	       (lambda ()
		 (newline)
		 (display name)
		 (display " ")
		 (display (current-time the-agenda))
		 (display " New-value = ")
		 (display (get-signal wire)))))

然后把上面的测试代码打包一下：

In [36]:
(define (start-unit-test-3-28)
  (define inverter-input (make-wire))
  (define inverter-output (make-wire))
  
  (probe 'inverter-input inverter-input)
  (probe 'inverter-output inverter-output)

  (inverter inverter-input inverter-output)

  (set-signal! inverter-input 1)

  (propagate)
  )


In [37]:
(start-unit-test-3-28)


inverter-input 4 New-value = 0
inverter-output 4 New-value = 0
inverter-input 4 New-value = 1
inverter-output 6 New-value = 1
inverter-output 6 New-value = 0

done

然后就照猫画老虎做个and-gate:

In [38]:
(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value
	   (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! a1 and-action-procedure)
  (add-action! a2 and-action-procedure)
  'ok)

(define (logical-and a1 a2)
  (cond ((and (not (= a1 0))
	     (not (= a1 1))) (error "Invalid signal" a1))
	((and (not (= a2 0))
	     (not (= a2 1))) (error "Invalid signal" a2))
	((and (= a1 1) (= a2 1)) 1)
	(else 0)))

(define and-gate-delay 3)


还有or-gate，这个就是题目的要求了：

In [39]:
(define (or-gate a1 a2 output)
  (define (or-action-procedure)
    (let ((new-value
	   (logical-or (get-signal a1) (get-signal a2))))
      (after-delay or-gate-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! a1 or-action-procedure)
  (add-action! a2 or-action-procedure)
  'ok)

(define (logical-or a1 a2)
  (cond ((and (not (= a1 0))
	     (not (= a1 1))) (error "Invalid signal" a1))
	((and (not (= a2 0))
	     (not (= a2 1))) (error "Invalid signal" a2))
	((or (= a1 1) (= a2 1)) 1)
	(else 0)))


(define or-gate-delay 5)


再测试一下 and-gate和or-gate:

In [40]:
(define (and-gate-test input-1-value input-2-value)
  (define input-1 (make-wire))
  (define input-2 (make-wire))
  (define output (make-wire))
  
  (and-gate input-1 input-2 output)
  
  (set-signal! input-1 input-1-value)
  (set-signal! input-2 input-2-value)
  
  (propagate)
  
  (get-signal output)
)


In [41]:
(list (and-gate-test 0 0) (and-gate-test 0 1) (and-gate-test 1 0) (and-gate-test 1 1))

(0 0 0 1)

In [42]:
(define (or-gate-test input-1-value input-2-value)
  (define input-1 (make-wire))
  (define input-2 (make-wire))
  (define output (make-wire))
  
  (or-gate input-1 input-2 output)
  
  (set-signal! input-1 input-1-value)
  (set-signal! input-2 input-2-value)
  
  (propagate)
  
  (get-signal output)
)


In [43]:
(list (or-gate-test 0 0) (or-gate-test 0 1) (or-gate-test 1 0) (or-gate-test 1 1))

(0 1 1 1)

到现在开始才是关于半加器的代码：

In [44]:
(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s)
    'ok))




再定义一个测试函数：

In [50]:
(define (half-adder-test a-value b-value)
  (define input-1 (make-wire))
  (define input-2 (make-wire))
  (define s-output (make-wire))
  (define c-output (make-wire))
  
  
  (half-adder input-1 input-2 s-output c-output)
  
  (set-signal! input-1 a-value)
  (set-signal! input-2 b-value)
  
  (propagate)
  
  (list (get-signal s-output) (get-signal c-output))
)


In [51]:
(list (half-adder-test 0 0) (half-adder-test 0 1) 
      (half-adder-test 1 0) (half-adder-test 1 1))

((0 0) (1 0) (1 0) (0 1))

然后全加器就是由两个半加器构成的，额外加个或门：

In [55]:


(define (full-adder a b c-in sum c-out)
  (let ((s (make-wire))
	(c1 (make-wire))
	(c2 (make-wire)))
    (half-adder b c-in s c1)
    (half-adder a s sum c2)
    (or-gate c1 c2 c-out)
    'ok))



In [56]:
(define (full-adder-test a-value b-value c-in-value)
  (define input-1 (make-wire))
  (define input-2 (make-wire))
  (define input-c (make-wire))
  (define s-output (make-wire))
  (define c-output (make-wire))
  
  
  (full-adder input-1 input-2 input-c s-output c-output)
  
  (set-signal! input-1 a-value)
  (set-signal! input-2 b-value)
  (set-signal! input-c c-in-value)
  
  (propagate)
  
  (list (get-signal s-output) (get-signal c-output))
)


In [59]:
(list (full-adder-test 0 0 0)
      (full-adder-test 1 0 0)
      (full-adder-test 0 1 0)
      (full-adder-test 1 1 0)
      (full-adder-test 0 0 1)
      (full-adder-test 1 0 1)
      (full-adder-test 0 1 1)
      (full-adder-test 1 1 1))

((0 0) (1 0) (1 0) (0 1) (1 0) (0 1) (0 1) (1 1))

要算时延就需要把最长的路线的时延叠加起来。

当然我们也可以通过probe来测试：

In [60]:
(define (full-adder-test a-value b-value c-in-value)
  (define input-1 (make-wire))
  (define input-2 (make-wire))
  (define input-c (make-wire))
  (define s-output (make-wire))
  (define c-output (make-wire))
  
  
  (full-adder input-1 input-2 input-c s-output c-output)
  
  (set-signal! input-1 a-value)
  (set-signal! input-2 b-value)
  (set-signal! input-c c-in-value)
  
  (probe 's-output s-output)
  
  (propagate)
  
  (list (get-signal s-output) (get-signal c-output))
)


In [61]:
(full-adder-test 0 0 1)


s-output 308 New-value = 0
s-output 324 New-value = 1

(1 0)

In [62]:
(full-adder-test 0 0 1)


s-output 324 New-value = 0
s-output 340 New-value = 1

(1 0)

需要26个时延